In [1]:
# Import packages
import torch
import transformers
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv(r"..\archive\SMS_train.csv", encoding='latin')
test = pd.read_csv(r"..\archive\SMS_test.csv", encoding='latin')

In [3]:
train

,S. No.,Message_body,Label
0,1,Rofl. Its true to its name,Non-Spam
1,2,The guy did some bitching but I acted like i'd...,Non-Spam
2,3,"Pity, * was in mood for that. So...any other s...",Non-Spam
3,4,Will ü b going to esplanade fr home?,Non-Spam
4,5,This is the 2nd time we have tried 2 contact u...,Spam
...,...,...,...
952,953,hows my favourite person today? r u workin har...,Non-Spam
953,954,How much you got for cleaning,Non-Spam
954,955,Sorry da. I gone mad so many pending works wha...,Non-Spam
955,956,Wat time ü finish?,Non-Spam


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 957 entries, 0 to 956
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   S. No.        957 non-null    int64 
 1   Message_body  957 non-null    object
 2   Label         957 non-null    object
dtypes: int64(1), object(2)
memory usage: 22.6+ KB


In [5]:
train.Message_body = train.Message_body.astype('str')

In [6]:
model_name = "bert-base-uncased"
max_length = 512

In [7]:
label_names = list(set(train.Label))
label_names

['Spam', 'Non-Spam']

In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit_transform(label_names)

array([1, 0], dtype=int64)

In [9]:
train_label = le.transform(train.Label)

In [10]:
train_X, val_X, train_y, val_y = train_test_split(train.Message_body, train_label, test_size=0.2, random_state=1234)

In [11]:
train_X

701             I'm really sorry I lit your hair on fire
362          I will reach ur home in  &lt;#&gt;  minutes
847    Aiyo... Her lesson so early... I'm still sleep...
330          Jay wants to work out first, how's 4 sound?
378    Our records indicate u maybe entitled to 5000 ...
                             ...                        
204    A £400 XMAS REWARD IS WAITING FOR YOU! Our com...
53     Miles and smiles r made frm same letters but d...
294                        Oh yeah clearly it's my fault
723                I need details about that online job.
815    Your weekly Cool-Mob tones are ready to downlo...
Name: Message_body, Length: 765, dtype: object

In [12]:
train_y

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,

In [13]:
tokenizer = BertTokenizerFast.from_pretrained(model_name, do_lower_case=True)

In [14]:
# and pad with 0's when less than `max_length`
train_encodings = tokenizer(list(train_X), truncation=True, padding=True, max_length=max_length)
val_encodings = tokenizer(list(val_X), truncation=True, padding=True, max_length=max_length)

In [15]:
# to make the encoding as torch dataset
class NewsGroupsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]]).type(torch.LongTensor)
        return item

    def __len__(self):
        return len(self.labels)

# convert tokenized data into a torch Dataset
train_dataset = NewsGroupsDataset(train_encodings, train_y)
val_dataset = NewsGroupsDataset(val_encodings, val_y)

In [16]:
# loading the model
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(label_names)).to("cuda")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [17]:
training_args = TrainingArguments(
    output_dir='../results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=50,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='../logs',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    logging_steps=100,               # log & save weights each logging_steps
    save_steps=100,
    evaluation_strategy="steps",     # evaluate each `logging_steps`
)

In [18]:
from sklearn.metrics import accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    # calculate accuracy using sklearn's function
    acc = accuracy_score(labels, preds)
    return {
      'accuracy': acc,
    }

In [19]:
trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)

In [20]:
trainer.train()

***** Running training *****
  Num examples = 765
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 288


Step,Training Loss,Validation Loss,Accuracy
100,0.180800,0.001002,1.000000
200,0.056300,0.003956,1.000000


***** Running Evaluation *****
  Num examples = 192
  Batch size = 16
Saving model checkpoint to ../results\checkpoint-100
Configuration saved in ../results\checkpoint-100\config.json
Model weights saved in ../results\checkpoint-100\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 192
  Batch size = 16
Saving model checkpoint to ../results\checkpoint-200
Configuration saved in ../results\checkpoint-200\config.json
Model weights saved in ../results\checkpoint-200\pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ../results\checkpoint-100 (score: 0.001001750468276441).


TrainOutput(global_step=288, training_loss=0.0828456583743294, metrics={'train_runtime': 163.0296, 'train_samples_per_second': 14.077, 'train_steps_per_second': 1.767, 'total_flos': 165112465014000.0, 'train_loss': 0.0828456583743294, 'epoch': 3.0})